In [ ]:
# Введение
# При работе с непрерывными числовыми данными часто бывает полезно разделить (to bin)
# данные на несколько сегментов для дальнейшего анализа.
# Существует несколько терминов: сегментирование (bucketing), дискретное разделение (discrete binning),
#     дискретизация (discretization) или квантование (quantization).
#     Pandas поддерживает эти подходы с помощью функций cut и qcut.

# В этой статье говорится о том, как использовать функции pandas для преобразования непрерывных данных
# в набор дискретных сегментов. Как и многие функции pandas, cut и qcut могут показаться простыми,
# но у них есть множество возможностей.
# Думаю, даже опытные пользователи научатся нескольким приемам, которые будут полезны для анализа.

In [ ]:
# Биннинг (binning)
# Один из наиболее распространенных случаев биннинга выполняется при создании гистограммы.

# Рассмотрим пример с продажами. Гистограмма данных о продажах показывает,
# как непрерывный набор показателей продаж можно разделить на дискретные ячейки (например: 60 000–70 000 долларов США),
# а затем использовать их для группировки и подсчета учетных записей (account number).

In [ ]:
# импортируем необходимые модули:
import pandas as pd
import numpy as np
import seaborn as sns

# добавляем в графики красивости seaborn:
sns.set_style('whitegrid')

In [ ]:
raw_df = pd.read_excel('https://github.com/chris1610/pbpython/blob/master/data/2018_Sales_Total_v2.xlsx?raw=true')
raw_df.head()

In [ ]:
# Далее представлен код, который показывает, как суммировать информацию о продажах за 2018 год для группы клиентов.
# Это представление отображает количество клиентов, у которых продажи находятся в определенных диапазонах:

df = raw_df.groupby(['account number', 'name'])['ext price'].sum().reset_index()
df.head()

In [ ]:
df['ext price'].plot(kind='hist');

In [ ]:
# Существует множество других сценариев, в которых вы можете определить собственные интервалы (bins).

# В приведенном выше примере 8 интервалов с данными. Что, если бы мы захотели разделить наших клиентов на 3, 4 или 5 групп?

# Вот где в игру вступают qcut и cut.
# Эти функции кажутся похожими и выполняют аналогичные функции группирования, но имеют различия,
# которые могут сбивать с толку новых пользователей.

# Остальная часть статьи покажет, в чем их различия и как их использовать.

In [ ]:
# qcut
# В документации qcut описывается как "функция дискретизации на основе квантилей".
# По сути, это означает, что qcut пытается разделить базовые данные на интервалы равного размера.
# Функция определяет интервалы с использованием процентилей на основе распределения данных,
# а не фактических числовых границ интервалов.

In [ ]:
# Если вы ранее использовали функцию description, то уже встречали пример основных концепций, представленных qcut:

df['ext price'].describe()

# Запомните значения для 25%, 50% и 75% процентилей, поскольку мы напрямую рассматрим использование qcut.
# Самое простое использование qcut - определить количество квантилей и позволить pandas разделить данные.

In [ ]:
# В приведенном ниже примере мы просим pandas создать 4 группы одинакового размера:

pd.qcut(df['ext price'], q=4)

# В результате получается категориальный ряд (про категориальный тип данных в pandas см. тут),
# представляющий интервалы с продажами. Поскольку мы запросили квантили с q=4,
# поэтому интервалы соответствуют процентилям из функции describe.

# Типичным вариантом использования является сохранение результатов разбиения в исходном фрейме данных (dataframe)
# для будущего анализа.

In [ ]:
# В следующем примере мы создадим 4 интервала (также называемых квартилями)
# и 10 интервалов (также называемых децилями) и сохраним результаты обратно в исходный фрейм данных:

df['quantile_ex_1'] = pd.qcut(df['ext price'], q=4)
df['quantile_ex_2'] = pd.qcut(df['ext price'], q=10, precision=0)

df.head()

# Обратите внимание, как сильно различаются интервалы между quantile_ex_1 и quantile_ex_2. Я также добавил precision (точности),
# чтобы определить, сколько десятичных знаков использовать для вычисления точности интервала.

In [ ]:
# Можем посмотреть, как значения распределяются по интервалам с помощью value_counts:

df['quantile_ex_1'].value_counts()

In [ ]:
# Теперь для второго столбца:

df['quantile_ex_2'].value_counts()

In [ ]:
# Это иллюстрирует ключевую концепцию: в каждом случае в каждом интервале содержится равное количество наблюдений.

# Pandas за кулисами производит вычисления, чтобы определить ширину интервалов.
# Например, в quantile_ex_1 диапазон первого интервала составляет 74661.15, а второго - 9861.02 (110132 - 100271).

# Одна из проблем, связанных с этим подходом, заключается в том, что имена интервалов сложно объяснить конечному пользователю.

In [ ]:
# Например, если мы хотим разделить наших клиентов на 5 групп (также называемых квинтилями),
# как в случае с часто летающими авиакомпаниями, мы можем явно назвать интервалы, чтобы их было легче интерпретировать:

bin_labels_5 = ['Bronze', 'Silver', 'Gold', 'Platinum', 'Diamond']

df['quantile_ex_3'] = pd.qcut(df['ext price'],
                              q=[0, .2, .4, .6, .8, 1],
                              labels=bin_labels_5)
df.head()

В приведенном выше примере я сделал кое-что иначе.

# Во-первых, явно определил диапазон используемых квантилей: q=[0, .2, .4, .6, .8, 1],
#     а также задал метки labels=bin_labels_5 для использования при представлении интервалов.

In [ ]:
# Давайте проверим распределение:

df['quantile_ex_3'].value_counts()